# Math 8803 HW1

## Problem 1 - Connor-Stevens Model


The main differential equation for the Connor-Stevens Model is:

$$I(t) = C (\frac{dV}{dt}) + g_{L}(V - E_{L}) + g_{Na}(V - E_{Na}) + g_{K}(V - E_{K}) + g_{A}(V - E_{K})$$

#### Explanation of Parameters:

1. $ C (\frac{dV}{dt}) $ - We can think of the membrane as a capacitor and $V$ represents the voltage across the membrane. $C$ is capacitance or $C = \frac{q}{u} = \frac{q}{V}$ where $q$ is a charge and $v$ reprecents the voltage across a capacitor. Thus, $ C (\frac{dV}{dt}) $ is the charging current $I_{c}(t)$ that is applied on the membrane.

2. $E_{Na}$ - Battery Voltage for Sodium

3. $E_{K}$ - Battery Voltage for Potassium

4. $E_{L}$ - Battery Voltage for Unspecific Third Channel

5. $g_{Na} = \overline{g}_{Na} m^{3}h$ - Sodium Conductance. Here $\overline{g}_{Na}$ is a normalization constant, $m$ describes the activation/opening of the and $h$ describes the inactivation/blocking of the $Na^{+}$ channels.  

6. $g_{K} = \overline{g}_{k} n^{4}$ - Delayed Potassium Conductance, where $\overline{g}_{k}$ is a normalization constant and $n$ controls the $K^{+}$ gates. 

7. $g_{A} = \overline{g}_{A} A^{3}B$ - Transient Potassium Conductance, where $\overline{g}_{A}$ is a normalization factor, $A$ is an activation term, and $B$ is an inactivation term. 

8. $g_{L}$ - Leaky Battery Conductance

9. $V$ - Voltage across the cell membrane
